In [2]:
import torch
import pandas as pd
import joblib
from transformers import DistilBertTokenizer, DistilBertModel
from torch import nn
from sklearn.preprocessing import MinMaxScaler

# ================================
# LOAD ENCODERS & SCALER
# ================================
product_encoder = joblib.load("../NLP/product_encoder.pkl")
character_encoder = joblib.load("../NLP/character_encoder.pkl")

scaler = joblib.load("../NLP/feature_scaler.pkl")  # save during training


c:\Users\User\Desktop\Crochet_Project\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ================================
# MODEL DEFINITION (same as training)
# ================================
class CrochetTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.fc = nn.Linear(768, 5)

    def forward(self, ids, mask):
        x = self.bert(ids, attention_mask=mask).last_hidden_state[:,0]
        return self.fc(x)

In [4]:
# ================================
# LOAD MODEL
# ================================
model = CrochetTransformer()
model.load_state_dict(torch.load("../nlp/crochet_transformer.pt"))
model.eval()

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

Loading weights: 100%|██████████| 100/100 [00:00<00:00, 467.09it/s, Materializing param=transformer.layer.5.sa_layer_norm.weight]   
DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_projector.bias    | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 
vocab_layer_norm.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [5]:
# ================================
# INFERENCE FUNCTION
# ================================
def extract_attributes(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        preds = model(tokens["input_ids"], tokens["attention_mask"]).numpy()

    # reverse scaling
    preds = preds[0]   # shape (5,)

    # separate outputs
    product_pred = preds[0]
    height_pred = preds[1]
    width_pred = preds[2]
    colors_pred = preds[3]
    character_pred = preds[4]

    # inverse scale ONLY numeric values
    numeric = scaler.inverse_transform([[height_pred, width_pred, colors_pred]])[0]

    height = round(numeric[0])
    width = round(numeric[1])
    colors = round(numeric[2])

    # categorical decoding
    product_id = round(product_pred)
    character_id = round(character_pred)

    product = product_encoder.inverse_transform([product_id])[0]
    character = character_encoder.inverse_transform([character_id])[0]


    return {
        "product": product,
        "height": None if height == 0 else height,
        "width": None if width == 0 else width,
        "colors": None if colors == 0 else colors,
        "character": None if character == "0" else character
    }


In [6]:
# ================================
# QUICK TEST
# ================================
if __name__ == "__main__":
        text = input("\nYou: ")
        print(extract_attributes(text))

{'product': 'keychain', 'height': 18, 'width': 18, 'colors': 4, 'character': 'flowers'}
